In [5]:
import numpy as np
import cv2
from PIL import Image

In [8]:
import cv2
import numpy as np

# Load the original image and segment
original_image = cv2.imread('your_original_image.jpg', cv2.IMREAD_GRAYSCALE)
segment_image = cv2.imread('your_segment_image.jpg', cv2.IMREAD_GRAYSCALE)

# Initialize feature detector (ORB)
orb = cv2.ORB_create()

# Find keypoints and descriptors in both images
keypoints_original, descriptors_original = orb.detectAndCompute(original_image, None)
keypoints_segment, descriptors_segment = orb.detectAndCompute(segment_image, None)

# Initialize a brute force matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(descriptors_original, descriptors_segment)

# Sort matches based on distance
matches = sorted(matches, key=lambda x: x.distance)

# Extract matching keypoints
matched_points_original = np.float32([keypoints_original[match.queryIdx].pt for match in matches]).reshape(-1, 1, 2)
matched_points_segment = np.float32([keypoints_segment[match.trainIdx].pt for match in matches]).reshape(-1, 1, 2)

# Calculate transformation matrix (homography)
H, _ = cv2.findHomography(matched_points_segment, matched_points_original, cv2.RANSAC)

# Get the dimensions of the segment
segment_height, segment_width = segment_image.shape[:2]

# Define the corners of the segment in its own coordinate system
segment_corners = np.float32([[0, 0], [segment_width, 0], [segment_width, segment_height], [0, segment_height]]).reshape(-1, 1, 2)

# Transform the segment corners to the original image coordinate system
transformed_segment_corners = cv2.perspectiveTransform(segment_corners, H)

# Extract the location (bounding box) of the segment in the original image
min_x, min_y = np.int32(transformed_segment_corners.min(axis=0).ravel())
max_x, max_y = np.int32(transformed_segment_corners.max(axis=0).ravel())
segment_location = (min_x, min_y, max_x, max_y)

print(f"Location of the segment in the original image: {segment_location}")


[ WARN:0@702.109] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('your_original_image.jpg'): can't open/read file: check file path/integrity
[ WARN:0@702.110] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('your_segment_image.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'
